In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 21 13:38:32 2022

@author: comsee
"""
%load_ext autoreload
%autoreload 2

import os
os.chdir("../neuvisys-analysis")

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from src.spiking_network.network.neuvisys import SpikingNetwork

from src.spiking_network.analysis.network_statistics import (
    average_over_multiple_stimuli, 
)

In [ ]:
os.chdir(".../neuvisys-analysis/src")
home = "..."

SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 30


In [ ]:
networks_path3 = ""


path = networks_path3 + ""

spinet = SpikingNetwork(path)

In [ ]:
layer_id = 0
neuron_id = 2560 
neuron_z = 0
start = 125000
end = 2001000
reset_neuron_z = neuron_z
depth = spinet.l_shape[layer_id][2]
thickness = 1
theta = 0
thresh = 0
simulation_max = 2
if(layer_id==0):
    thresh=10
else:
    thresh=3
    
# fqcies = [1,2,3, 4, 5, 6, 7]
fqcies = [0, 1, 2, 3, 4]
angles = [0, 23, 45, 68, 90, 113, 135, 158]
surr_angles = [0,23,45,68,90,113,135,158]
rotations_surround = np.array([[0,90],[23,113],[45,135],[68,158], [90,0], [113, 23], [135, 45], [158,68]])

In [ ]:
fqcies = [0, 1, 2, 3, 4]
angles = [0, 23, 45, 68, 90, 113, 135, 158]
surr_angles = [0,23,45,68,90,113,135,158]
# surr_angles = [0, 1, 2, 3, 4]
rotations_surround = np.array([[0,90],[23,113],[45,135],[68,158], [90,0], [113, 23], [135, 45], [158,68]])

In [ ]:
folder = ".../statistics/suppression/OrientedSupprNO/"
list_files = [0]

In [ ]:
for val in list_files:
    print("Lat_value = {} !".format(val))
    average_over_multiple_stimuli(spinet,folder + str(val) + "/", angles, surr_angles, simulation_max, neuron_id, layer_id = 0, neuron_z = 0, max_depth = 64, thresh = 10, tuned_ori=False, lat_value = val)

In [ ]:
folder = ".../statistics/suppression/OrientedSupprNO/"

spikes_ag2 = [[]]
for i, angle in enumerate(angles):
    for surr_angle in angles:
        spikes_ag2[i].append(np.load(folder + str(0) + "/" + str(angle) + "/per_surround/" + str(surr_angle) + "/spikes_avg.npy"))
    if(i<7):
        spikes_ag2.append([])
spikes_ag2 = np.array(spikes_ag2)

In [ ]:


fig = plt.figure(0,figsize=(8,20))

ax = fig.add_axes([0,0,1,1])
real_new_fol = ".../statistics/suppression/"
folders = [real_new_fol + "OrientedSupprNO/", real_new_fol + "OrientedSupprLAT/",  real_new_fol + "OrientedSupprTD/", real_new_fol + "OrientedSupprALL/"]
max_ = 1
            
for col, folder in enumerate(folders):
    for index, val in enumerate(list_files):
        if(val==-1):
            continue
        spikes_ag = [[]]
        spikes_ag_std = [[]]
        for i, angle in enumerate(angles):
            for surr_angle in surr_angles:
                spikes_ag[i].append(np.load(folder + str(val) + "/" + str(angle) + "/per_surround/" + str(surr_angle) + "/spikes_avg.npy"))
                spikes_ag_std[i].append(np.load(folder + str(val) + "/" + str(angle) + "/per_surround/" + str(surr_angle) + "/spikes_stderr.npy"))
            if(i<7):
                spikes_ag.append([])
                spikes_ag_std.append([])
        spikes_ag = np.array(spikes_ag)
        spikes_ag_std = np.array(spikes_ag_std).squeeze(2)
        spikes_ag_min = spikes_ag - spikes_ag_std 
        spikes_ag_max = spikes_ag + spikes_ag_std 


        dicts = {}
        dicts_max = {}
        dicts_min = {}
        cter = {}
        keys = [[]]
        new_angles = [ [0, 23, 45, 68, 90, -67, -45, -22], 
                      [0 , 23, 45, 68, 90, 113, -45, -22], 
                      [0, 23, 45, 68, 90, 113, 135, -22], 
                      [0, 23,45, 68, 90, 113, 135, 158],
                       [180, 23, 45, 68, 90, 113, 135, 158],
                      [180, 203, 45, 68, 90, 113, 135, 158],
                      [180, 203, 225, 68, 90, 113, 135, 158],
                      [180, 203, 225, 248, 90, 113, 135, 158]]
        for i, angle in enumerate(angles):
            for surr_angle in new_angles[i]:
                keys[i].append((surr_angle-angle))
            if(i <= 6):
                keys.append([])
        for tab in keys:
            for i in tab:
                dicts[i]=0
                dicts_max[i]=0
                dicts_min[i]=0
                cter[i]=0
        for i, angle in enumerate(angles):
            for j, surr_angle in enumerate(new_angles[i]):
                dicts[(surr_angle-angle)]+=spikes_ag[i][j]
                dicts_max[(surr_angle-angle)]+=spikes_ag_max[i][j]
                dicts_min[(surr_angle-angle)]+=spikes_ag_min[i][j]
                cter[(surr_angle-angle)]+=1
        keys_ = dicts.keys()
        for e in keys_:
            dicts[e] = dicts[e]/cter[e]
            dicts_max[e] = dicts_max[e]/cter[e]
            dicts_min[e] = dicts_min[e]/cter[e]

        v = []
        v_max = []
        v_min = []
        for e in keys_:
            v.append(dicts[e])
            v_max.append(dicts_max[e])
            v_min.append(dicts_min[e])

        zipped_lists = zip(keys_, v)
        sorted_pairs = sorted(zipped_lists)
        zipped_lists_max = zip(keys_, v_max)
        sorted_pairs_max = sorted(zipped_lists_max)
        zipped_lists_min = zip(keys_, v_min)
        sorted_pairs_min = sorted(zipped_lists_min)


        tuples = zip(*sorted_pairs)
        tuples_max = zip(*sorted_pairs_max)
        tuples_min = zip(*sorted_pairs_min)


        keys_, v = [ list(tuple) for tuple in  tuples]
        _, v_max = [ list(tuple) for tuple in  tuples_max]
        _, v_min = [ list(tuple) for tuple in  tuples_min]

        keys_.append(-90)
        v.append(v[-1])
        v_max.append(v_max[-1])
        v_min.append(v_min[-1])
        zipped_lists = zip(keys_, v)
        zipped_lists_max = zip(keys_, v_max)
        zipped_lists_min = zip(keys_, v_min)

        sorted_pairs = sorted(zipped_lists)
        sorted_pairs_max = sorted(zipped_lists_max)
        sorted_pairs_min = sorted(zipped_lists_min)

        tuples = zip(*sorted_pairs)
        tuples_max = zip(*sorted_pairs_max)
        tuples_min = zip(*sorted_pairs_min)


        keys_, v = [ list(tuple) for tuple in  tuples]
        _, v_max = [ list(tuple) for tuple in  tuples_max]
        _, v_min = [ list(tuple) for tuple in  tuples_min]

        new_keys = []
        new_val = []
        new_val_max = []
        new_val_min = []
        for i, r in enumerate(keys_):
            if(abs(r)!=22 and abs(r)!=23 and abs(r)!=67 and abs(r)!=68):
                new_keys.append(r)
                new_val.append(v[i])
                new_val_max.append(v_max[i])
                new_val_min.append(v_min[i])

            elif(abs(r)==23):
                if(r >0):
                    new_keys.append(22.5)
                    new_val.append((v[i-1]+v[i])/2)
                    new_val_max.append((v_max[i-1]+v_max[i])/2)
                    new_val_min.append((v_min[i-1]+v_min[i])/2)

                else:
                    new_keys.append(-22.5)
                    new_val.append((v[i+1]+v[i])/2)
                    new_val_max.append((v_max[i+1]+v_max[i])/2)
                    new_val_min.append((v_min[i+1]+v_min[i])/2)

            elif(abs(r)==68):
                if(r >0):
                    new_keys.append(67.5)
                    new_val.append((v[i-1]+v[i])/2)
                    new_val_max.append((v_max[i-1]+v_max[i])/2)
                    new_val_min.append((v_min[i-1]+v_min[i])/2)

                else:
                    new_keys.append(-67.5)
                    new_val.append((v[i+1]+v[i])/2)
                    new_val_max.append((v_max[i+1]+v_max[i])/2)
                    new_val_min.append((v_min[i+1]+v_min[i])/2)




        ax.set_xlabel("Orientation difference (in degrees)", fontsize = 40)

        ax.set_ylabel("Normalized response", fontsize = 40)
        matplotlib.rc('xtick', labelsize=35) 
        matplotlib.rc('ytick', labelsize=35) 
        matplotlib.rcParams['legend.fontsize'] = 35

        new_val = np.array(new_val)
        new_val_max = np.array(new_val_max)
        new_val_min = np.array(new_val_min)
        if(col==2):
            ax.plot(new_keys, new_val/max_, "b-", linewidth = 3.0, label = "LOCAL + TD")
        if(col==1):
            ax.plot(new_keys, new_val/max_, color="#FFA500", linewidth = 3.0,label = "LOCAL + DIST")
        if(col==0):
            max_ = np.max(new_val)
            print(max_)
            ax.plot(new_keys, new_val/max_, "k-", linewidth = 3.0,label = "LOCAL")
        if(col==3):
            ax.plot(new_keys, new_val/max_, "r-", linewidth = 3.0,label = "LOCAL + DIST + TD")
        print(1-(new_val[4]/new_val[0]))

        for e in range(len(new_val)):
            if(col==2):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "b--")
            if(col==1):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], color="#FFA500",linestyle="dashed")
            if(col==0):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "k--")
            if(col==3):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "r--")

    plt.xticks([-90, -45,  0,  45,  90])

plt.legend(loc='center')   
plt.show()

In [ ]:
folder = ".../statistics/suppression/CrossSupprNOINHIB/"

spikes_ag2 = [[]]
for i, angle in enumerate(angles):
    for surr_angle in angles:
        spikes_ag2[i].append(np.load(folder + str(0) + "/" + str(angle) + "/per_surround/" + str(surr_angle) + "/spikes_avg.npy"))
    if(i<7):
        spikes_ag2.append([])
spikes_ag2 = np.array(spikes_ag2)

In [ ]:
fig = plt.figure(0,figsize=(8,20))

ax = fig.add_axes([0,0,1,1])

real_new_fol = ".../statistics/suppression/"
folders = [real_new_fol + "CrossSupprNOINHIB/", real_new_fol + "CrossSupprNO/",  real_new_fol + "CrossSupprLAT-TD/", real_new_fol + "CrossSupprALL/"]
max_ = 1
for col, folder in enumerate(folders):

    for index, val in enumerate(list_files):
        if(val==-1):
            continue
        spikes_ag = [[]]
        spikes_ag_std = [[]]
        for i, angle in enumerate(angles):
            for surr_angle in surr_angles:
                spikes_ag[i].append(np.load(folder + str(val) + "/" + str(angle) + "/per_surround/" + str(surr_angle) + "/spikes_avg.npy"))
                spikes_ag_std[i].append(np.load(folder + str(val) + "/" + str(angle) + "/per_surround/" + str(surr_angle) + "/spikes_stderr.npy"))
            if(i<7):
                # print(i)
                spikes_ag.append([])
                spikes_ag_std.append([])
        spikes_ag = np.array(spikes_ag)
        spikes_ag_std = np.array(spikes_ag_std).squeeze(2)
        spikes_ag_min = spikes_ag - spikes_ag_std 
        spikes_ag_max = spikes_ag + spikes_ag_std 

        dicts = {}
        dicts_max = {}
        dicts_min = {}
        cter = {}
        keys = [[]]
        new_angles = [ [0, 23, 45, 68, 90, -67, -45, -22], 
                      [0 , 23, 45, 68, 90, 113, -45, -22], 
                      [0, 23, 45, 68, 90, 113, 135, -22], 
                      [0, 23,45, 68, 90, 113, 135, 158],
                       [180, 23, 45, 68, 90, 113, 135, 158],
                      [180, 203, 45, 68, 90, 113, 135, 158],
                      [180, 203, 225, 68, 90, 113, 135, 158],
                      [180, 203, 225, 248, 90, 113, 135, 158]]
        for i, angle in enumerate(angles):
            for surr_angle in new_angles[i]:
                keys[i].append((surr_angle-angle))
            if(i <= 6):
                keys.append([])
        for tab in keys:
            for i in tab:
                dicts[i]=0
                dicts_max[i]=0
                dicts_min[i]=0
                cter[i]=0
        for i, angle in enumerate(angles):
            for j, surr_angle in enumerate(new_angles[i]):
                dicts[(surr_angle-angle)]+=spikes_ag[i][j]
                dicts_max[(surr_angle-angle)]+=spikes_ag_max[i][j]
                dicts_min[(surr_angle-angle)]+=spikes_ag_min[i][j]
                cter[(surr_angle-angle)]+=1
        keys_ = dicts.keys()
        for e in keys_:
            dicts[e] = dicts[e]/cter[e]
            dicts_max[e] = dicts_max[e]/cter[e]
            dicts_min[e] = dicts_min[e]/cter[e]

        v = []
        v_max = []
        v_min = []
        for e in keys_:
            v.append(dicts[e])
            v_max.append(dicts_max[e])
            v_min.append(dicts_min[e])

        zipped_lists = zip(keys_, v)
        sorted_pairs = sorted(zipped_lists)
        zipped_lists_max = zip(keys_, v_max)
        sorted_pairs_max = sorted(zipped_lists_max)
        zipped_lists_min = zip(keys_, v_min)
        sorted_pairs_min = sorted(zipped_lists_min)


        tuples = zip(*sorted_pairs)
        tuples_max = zip(*sorted_pairs_max)
        tuples_min = zip(*sorted_pairs_min)


        keys_, v = [ list(tuple) for tuple in  tuples]
        _, v_max = [ list(tuple) for tuple in  tuples_max]
        _, v_min = [ list(tuple) for tuple in  tuples_min]

        keys_.append(-90)
        v.append(v[-1])
        v_max.append(v_max[-1])
        v_min.append(v_min[-1])
        zipped_lists = zip(keys_, v)
        zipped_lists_max = zip(keys_, v_max)
        zipped_lists_min = zip(keys_, v_min)

        sorted_pairs = sorted(zipped_lists)
        sorted_pairs_max = sorted(zipped_lists_max)
        sorted_pairs_min = sorted(zipped_lists_min)

        tuples = zip(*sorted_pairs)
        tuples_max = zip(*sorted_pairs_max)
        tuples_min = zip(*sorted_pairs_min)


        keys_, v = [ list(tuple) for tuple in  tuples]
        _, v_max = [ list(tuple) for tuple in  tuples_max]
        _, v_min = [ list(tuple) for tuple in  tuples_min]

        new_keys = []
        new_val = []
        new_val_max = []
        new_val_min = []
        for i, r in enumerate(keys_):
            if(abs(r)!=22 and abs(r)!=23 and abs(r)!=67 and abs(r)!=68):
                new_keys.append(r)
                new_val.append(v[i])
                new_val_max.append(v_max[i])
                new_val_min.append(v_min[i])

            elif(abs(r)==23):
                if(r >0):
                    new_keys.append(22.5)
                    new_val.append((v[i-1]+v[i])/2)
                    new_val_max.append((v_max[i-1]+v_max[i])/2)
                    new_val_min.append((v_min[i-1]+v_min[i])/2)

                else:
                    new_keys.append(-22.5)
                    new_val.append((v[i+1]+v[i])/2)
                    new_val_max.append((v_max[i+1]+v_max[i])/2)
                    new_val_min.append((v_min[i+1]+v_min[i])/2)

            elif(abs(r)==68):
                if(r >0):
                    new_keys.append(67.5)
                    new_val.append((v[i-1]+v[i])/2)
                    new_val_max.append((v_max[i-1]+v_max[i])/2)
                    new_val_min.append((v_min[i-1]+v_min[i])/2)

                else:
                    new_keys.append(-67.5)
                    new_val.append((v[i+1]+v[i])/2)
                    new_val_max.append((v_max[i+1]+v_max[i])/2)
                    new_val_min.append((v_min[i+1]+v_min[i])/2)

        ax.set_xlabel("Orientation difference (in degrees)", fontsize = 40)
        ax.set_ylabel("Normalized response", fontsize = 40)
        matplotlib.rc('xtick', labelsize=35) 
        matplotlib.rc('ytick', labelsize=35) 
        matplotlib.rcParams['legend.fontsize'] = 35
        new_val = np.array(new_val)
        if(col==2):
            ax.plot(new_keys, new_val/max_, "m-", linewidth = 3.0, label = "DIST + TD")
        if(col==1):
            ax.plot(new_keys, new_val/max_, "k-", linewidth = 3.0,label = "LOCAL")
        if(col==0):
            max_ = np.max(new_val)
            ax.plot(new_keys, new_val/max_, "g-", linewidth = 3.0,label = "NO INHIBITION")
        if(col==3):
            ax.plot(new_keys, new_val/max_, "r-", linewidth = 3.0,label = "LOCAL + DIST + TD")
        print(1-(new_val[4]/new_val[0]))

        for e in range(len(new_val)):
            if(col==2):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "m--")
            if(col==1):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "k--")
            if(col==0):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "g--")
            if(col==3):
                ax.plot([new_keys[e], new_keys[e]], [new_val_max[e]/max_, new_val_min[e]/max_], "r--")


    plt.xticks([-90, -45,  0,  45,  90])

plt.legend(loc='center')   
plt.show()